In [1]:
'''
Conditions on the clusters (Y)_s: 

1, Every v has its r-nbhd completely contained in EXACTLY one cluster. 
2. Overlap #define as max no' of clusters that contain a single v, less than beta times the beta'th root of n 
3. For every element in some ker(~) the r-nbhd of that element is contained in ~ .  
4. All the kers are disjoint and yet cover the entire set.

Invariants through _a single phase_

1. All Ys in a phase disjoint. 
2. All kerZs in a phase disjoint.
3. Z can overlap. 
4. For every element in some ker(~) the r-nbhd of that element is contained in ~ .  
5. The union of all Ys is a subset of the union of all kerZs . 
6. The ker(Z), Z relationship and the ker(Y), Y relationship are maintained -- as well as the Y_s, kerZ_s relationship mentioned above.

'''

"\nConditions on the clusters (Y)_s: \n\n1, Every v has its r-nbhd completely contained in EXACTLY one cluster. \n2. Overlap #define as max no' of clusters that contain a single v, less than beta times the beta'th root of n \n3. For every element in some ker(~) the r-nbhd of that element is contained in ~ .  \n4. All the kers are disjoint and yet cover the entire set.\n\nInvariants through _a single phase_\n\n1. All Ys in a phase disjoint. \n2. All kerZs in a phase disjoint.\n3. Z can overlap. \n4. For every element in some ker(~) the r-nbhd of that element is contained in ~ .  \n5. The union of all Ys is a subset of the union of all kerZs . \n6. The ker(Z), Z relationship and the ker(Y), Y relationship are maintained -- as well as the Y_s, kerZ_s relationship mentioned above.\n\n"

In [2]:
import sys 
sys.path.append('./lib') 

In [3]:
import networkx as nx 
import plotly.graph_objects as go 
from lib.graphfig import * 
from lib.bfs import * 
from lib.even_shiloach import *

In [101]:
def functional_get_cluster(R:set[int],U:set[int],v:int,Y_s:set[set[int]],G:nx.Graph,r:int,beta:int|float):
    
    n = len(G.nodes)
    if len(Y_s) == 0:
        G_notin_Y = G.copy()
    else:
        G_notin_Y = nx.induced_subgraph(G,set(G.nodes)-set.union(*Y_s))
    
    ker_z = set([v,])
    T_z = BFSAlgo(G,ker_z,r)
    
    z = set(T_z.bfs_graph.nodes)
    
    for v_z in z:  
        assert T_z.bfs_graph.nodes[v_z]['level'] <= r 

    cluster_iters = 0
    while True: 
        ker_y, y = ker_z.copy(), z.copy()
        T_z = BFSAlgo(G_notin_Y,y,2*r)
        z = set(T_z.bfs_graph.nodes)
        ker_z = set([v for v in U.intersection(z) if  T_z.bfs_graph.nodes[v]['level']  <= r ]) 

        n_root_beta = n**(1/beta) 
        lenR_root_beta = len(R)**(1/beta)

        A1 = len(z)
        A2 = n_root_beta*len(y) 
        B1 = len(ker_z) 
        B2 = len(ker_y)*lenR_root_beta
        C1 = sum(dict(G.degree(z)).values()) 
        C2 = sum(dict(G.degree(y)).values())*n_root_beta 

        # print("L3")
        
        # print(f"Cluster Iteration {cluster_iters}:")
        # print(f"len(z) = {A1}")
        # print(f"len(y) = {len(y)}") 
        # print(f"A1 = {round(A1,2)}, A2 = {round(A2,2)}, B1 = {round(B1,2)}, B2 = {round(B2,2)}, C1 = {round(C1,2)}, C2 = {round(C2,2)}\n\n")
        # yield None 
        if(A1 <= A2 and B1 <= B2 and C1 <= C2):
            # yield True 
            break 
        cluster_iters += 1
    return (ker_y, y, ker_z, z)

In [ ]:
def functional_get_cover(R:set[int], G:nx.Graph, r:int, beta:int):
    U = R.copy() 
    ker_R = set() 
    Y_s, Z_s = [], []
    i = 0
    while len(U) > 0:
        print(i, len(U))
        i += 1
        v = next(iter(U))
        ker_y, y, ker_z, z = functional_get_cluster(R,U,v,Y_s,G,r,beta)
        ker_R = ker_R.union(ker_z)
        U = U.difference(ker_z)
        Y_s.append(y)
        Z_s.append(z)
    return (ker_R, Y_s, Z_s)

In [184]:
for p_search in reversed(range(1,31)):
    
    base_graph =  nx.fast_gnp_random_graph(10000,p_search*1e-4)
    try:
        assert nx.is_connected(base_graph)
    except AssertionError:
        print(p_search)

7
6
5
4
3
2
1


In [103]:
def get_connected_gnp_graph(n, lower_bound_n, p): 
    pre_base_graph =  nx.fast_gnp_random_graph(n,p)
    cc_nodes_lst = list(nx.connected_components(pre_base_graph))
    cc_lens_lst = [len(i) for i in cc_nodes_lst]
    idx = cc_lens_lst.index(max(cc_lens_lst)) 
    base_graph = nx.induced_subgraph(pre_base_graph,cc_nodes_lst[idx]).copy()
    try:
        assert len(base_graph.nodes) > lower_bound_n
    except AssertionError:
        print(len(base_graph.nodes)) 
        print([len(i) for i in nx.connected_components(pre_base_graph)])
        raise AssertionError

    return base_graph

In [73]:
[len(i) for i in list(nx.connected_components(base_graph))]

[3814]

In [88]:
from tqdm import tqdm

In [91]:
for _ in tqdm(range(100)):
    base_graph = get_connected_gnp_graph(20000,5000,0.00006)

100%|██████████| 100/100 [00:11<00:00,  8.79it/s]


In [118]:
R = set(base_graph.nodes)
U = R.copy()
v = random.choice(list(base_graph.nodes))
Y_s = []
G = base_graph
r = 4
beta = 4
print(len(base_graph.nodes))
# out = functional_get_cluster(R,U,v,Y_s,G,r,beta)
out = functional_get_cover(R,G,r,beta)

6134
0 6134
1 5879
2 5662
3 5384
4 5121
5 4175
6 4067
7 4059
8 3885
9 3768
10 3559
11 3377


KeyboardInterrupt: 

In [116]:
def functional_get_cover_all(G,r,beta): 
    R = set(G.nodes)
    CHI = []
    while len(R) > 0:
        ker_R, Y_s, Z_s =  functional_get_cover(R,G,r,beta)
        CHI.extend(Y_s)
        R = R.difference(ker_R)
    return CHI

In [119]:
CHI = functional_get_cover_all(base_graph,r,beta)

0 6134
1 5879
2 5662
3 5384
4 5121
5 4175
6 4067
7 4059
8 3885
9 3768
10 3559
11 3377
12 2187
13 2100
14 2077
15 2029
16 2026
17 2020
18 1971
19 1890
20 1840
21 1768
22 1712
23 1617
24 1564
25 1560
26 1554
27 1541
28 1504
29 1489
30 1414
31 1400
32 1399
33 1353
34 1277
35 1240
36 1234
37 1186
38 1109
39 1105
40 1088
41 1083
42 1048
43 1009
44 977
45 933
46 918
47 916
48 838
49 831
50 828
51 823
52 789
53 785
54 768
55 738
56 703
57 669
58 667
59 621
60 620
61 546
62 543
63 527
64 483
65 477
66 458
67 457
68 413
69 407
70 388
71 385
72 356
73 348
74 343
75 326
76 318
77 310
78 307
79 300
80 298
81 293
82 292
83 280
84 272
85 261
86 256
87 219
88 215
89 204
90 191
91 186
92 181
93 165
94 163
95 146
96 145
97 133
98 130
99 129
100 126
101 119
102 117
103 112
104 107
105 103
106 98
107 88
108 86
109 80
110 76
111 67
112 65
113 63
114 60
115 57
116 56
117 55
118 52
119 50
120 46
121 39
122 38
123 37
124 33
125 30
126 26
127 25
128 24
129 21
130 17
131 16
132 13
133 9
134 7
135 6
136 4
137 2

In [108]:
ker_R, Y_s, Z_s = out 

In [111]:
from itertools import product

In [112]:
for y_1, y_2 in tqdm(product(Y_s,Y_s)): 
    assert isinstance(y_1,set) and isinstance(y_2,set) 
    if not (y_1 is y_2): 
        assert y_1.isdisjoint(y_2)

17956it [00:00, 1569051.91it/s]


In [115]:
len(Y_s)

134